# PA2 Discussion
## DSC 204a, Winter 2024

Notebook Setup: This notebook is best run on the `ray-notebook` server setup on DataHub, where all the dependencies have been installed for you.

## A Simple Introduction to Ray Actors

In [1]:
import ray
import time
import warnings
warnings.filterwarnings("ignore")
ray.init()

@ray.remote
class Counter:
    def __init__(self):
        self.value = 0

    def increment(self):
        # simulate longer execution time
        time.sleep(1)

        self.value += 1
        return self.value

    def get_counter(self):
        return self.value

# Create an actor instance from this class.
counter = Counter.remote()

2025-05-05 11:05:52,078	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [2]:
counter.value # doesn't work directly - access actor state through methods!

AttributeError: 'ActorHandle' object has no attribute 'value'

In [ ]:
counter.increment.remote()

ObjectRef(ae46b8beecd25f3adbab2eb007d7f15f358cf4380100000001000000)

In [3]:
start = time.time()
for _ in range(15):
    counter.increment.remote() # increment the counter. Note that each method is a Ray task now!
end = time.time()
print("Time :", end - start)

Time : 0.003648042678833008


In [4]:
ray.get(counter.get_counter.remote()) 

15

Wait, shouldn't the time be 15 s? Well, these increment method calls are now Ray tasks, and are thus executed asynchronously!

All the method calls of an actor are executed in order i.e serially. Thus, the `get_counter` call will wait until all the previous `increment` calls complete execution (which is why you will see that this final `get_counter` call takes some time).

## Multiprocessing in (Native) Python 

Multiprocessing in native Python is a slightly more complex task, especially when we need to coordinate between processes. 

We introduce 2 ways to communicate between processes:

1. Shared Memory (see `mp_shmem.py`): The naive, less optimal way to coordinate between processes.
2. Events (see `mp_event.py`): The more optimal way to coordinate between processes.

More advaned (optional):

3. Queues (see `mp_queue.py`)
4. Conditional Variables (see `mp_condition.py`)

Note: In your homework, you only need to pick one of the above methods for your implementation. No need to 

### Method 1: Shared Memory

See: [mp_naive.py](./mp_naive.py)

This is the most basic (and least efficient) coordination method. Processes communicate through a shared dictionary, and one process constantly polls for a value using time.sleep().
- ✅ Simple to understand
- ❌ Wastes CPU cycles (busy waiting)
- ❌ Not scalable for more complex tasks


```python
import multiprocessing as mp
from multiprocessing import Process
import time

def task_a(ctx):
    ctx['a'] = 1

def task_b(ctx):
    while 'a' not in ctx:
        time.sleep(0.01)
    ctx['b'] = 2
```

In [4]:
!python mp_shmem.py

[Task B] Enter Task B[Task A] Enter Task A

[Task A] Before waiting - shared_dict: {}
[Task B] Before waiting - shared_dict: {}
[Task A] After waiting - shared_dict: {'a': 1}
[Task A] Exit Task A
[Task B] After waiting - shared_dict: {'a': 1}
[Task B] After updating - shared_dict: {'a': 1, 'b': 2}
[Task B] Exit Task B


### Method 2: Events

See: [mp_event.py](./mp_event.py)

This is a more efficient synchronization mechanism. One process sets an Event once it’s ready, and the other waits for it — no polling needed.

- ✅ More efficient than polling
- ❌ Less flexible than shared memory

```python
import multiprocessing as mp
from multiprocessing import Process
import time

def task_a(ctx, evt):
    ctx['a'] = 1
    evt.set()

def task_b(ctx, evt):
    evt.wait()
    ctx['b'] = 2
```


In [5]:
! python mp_event.py

[Task A] Enter Task A
[Task B] Enter Task B; waiting for Task A…
[Task A] Set ctx['a']=1; signalling Task B
[Task A] Exit Task A
[Task B] Detected ctx: {'a': 1}
[Task B] Exit Task B


### (Optional) Method 3: Queue

See: [mp_queue.py](./mp_queue.py)

Best suited for producer-consumer or pipeline models. A process can put() data into a Queue, and another can get() it, all in a thread-safe, blocking fashion.
- ✅ Easy to pass arbitrary data
- ✅ Scales well across multiple producers/consumers
- ✅ Clean decoupling of process logic
- ⚠️ Not as efficient as Events 
- ⚠️ Not as flexible as simple shared-memory based method (as it needs to pass the `Queue` around)

```python
import multiprocessing as mp
from multiprocessing import Process
q = mp.Queue()

def task_a(q):
    result = {"a": 1}
    q.put(result)

def task_b(q):
    result = q.get()
```



In [6]:
! python mp_queue.py

[Task A] Enter Task A
[Task A] Sending {'a': 1}
[Task A] Exit Task A
[Task B] Enter Task B; waiting for message
[Task B] Received {'a': 1}
[Task B] Exit Task B


### (Optional) Method 4: Conditional Variables

See: [mp_condition.py](./mp_condition.py)

Use this when you want a process to wait for a specific condition to become true. It’s more flexible than an Event and can handle more complex dependencies.
- ✅ Fine-grained control over when to wake up
- ⚠️ Requires careful handling of locking and condition logic


In [7]:
!python mp_condition.py

[Task A] Enter Task A
[Task A] Set ctx['a']=1; notifying all
[Task A] Exit Task A
[Task B] Enter Task B; waiting for predicate
[Task B] Detected ctx: {'a': 1}
[Task B] Exit Task B
